In [1]:
########################################################################################
########################################################################################

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

########################################################################################
########################################################################################

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import random
import keras
import os
from PIL import Image
import PIL
import cv2
import wordcloud
from matplotlib import colors

from tqdm import tqdm
import json
import shutil
from glob import glob

########################################################################################
##########################       Augmentation d'image       ############################

try :
    import imgaug    #  import imaug as ia
    import imgaug as ia
except :
    !pip install imgaug
    import imgaug     #  import imaug as ia
    import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
from imgaug.augmentables.batches import UnnormalizedBatch

########################################################################################
########################################################################################
from sklearn.base import BaseEstimator,TransformerMixin

########################################################################################
########################################################################################

try :
    from tensorflow.keras.preprocessing.image import image_utils
except :
    from tensorflow.keras.preprocessing import image as image_utils
    
from tensorflow.python.keras.losses import CategoricalCrossentropy, BinaryCrossentropy, categorical_crossentropy
from tensorflow.python.keras import backend as K
import tensorflow.compat.v2 as tf

########################################################################################
########################################################################################

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from sklearn import preprocessing, manifold, metrics
from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow.python.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.python.keras import backend as K

########################################################################################
########################################################################################

import seaborn as sns
#%matplotlib inline
#%matplotlib notebook
sns.set()

########################################################################################
########################################################################################

#!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
#!pip3 install tensorflow-hub
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
def plot_confusion_matrix(Y_true , Y_predict, title="Matrice de confusion", cmap="hot_r", figsize = (12,8), tic_rot= (0,0)):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        warnings.filterwarnings('ignore')
        Y_true , Y_predict = np.asarray(Y_true), np.asarray(Y_predict)
        df_confusion = pd.crosstab(Y_true, Y_predict, rownames=['True Labels\n'], colnames=['\nPredicted Labels'])
        plt.figure(figsize=figsize)
        sns.heatmap(df_confusion, cmap=cmap, annot =True) # imshow
        plt.title(title,size =3*figsize[0])
        #plt.colorbar()
        tick_marks = np.arange(len(df_confusion.columns)) +0.5
        plt.xticks(tick_marks, df_confusion.columns, rotation=tic_rot[0], size=1.5*figsize[0])
        plt.yticks(tick_marks, df_confusion.index, rotation=tic_rot[1], size = 1.5*figsize[0])
        #plt.tight_layout()
        plt.ylabel(df_confusion.index.name, size = 2*figsize[0] )
        plt.xlabel(df_confusion.columns.name, size = 2*figsize[0])
        plt.show()

def get_train_test_index( datas , label = "label", nombre=[200,200,200] ) :
    data = datas.copy()
    shape_0 = int(data.shape[0]/data[label].nunique())
    # si les valeurs sont inférieures à zéro
    if nombre[0]<1 and nombre[1]<1 and nombre[2]<1 :
            nombre[0] = int( nombre[0]*shape_0 )
            nombre[1] = int( nombre[1]*shape_0 )
            nombre[2] = int( nombre[2]*shape_0 )
    
    # Si la somme des valeurs est différente de la taille des données
    if np.sum(nombre) != shape_0 :
        nombre[1] = (shape_0 - nombre[0])//2
        nombre[2] = nombre[1]
        nombre[0] = shape_0 - 2*nombre[1]
        
    #for i in range(3) : print(nombre[i])
    #print(np.sum(nombre)*data[label].nunique())
    
    idx = [[] for i in range(len(nombre)) ]
    for i in range(len(nombre)) :
        if i != len(nombre) -1 :
            for cat in data[label].unique() : 
                sub_data = data[ data[label] == cat]
                for j in random.sample( sorted(sub_data.index.values), nombre[i] ) :
                    idx[i].append(j)
            data.drop(index=idx[i] , inplace=True)
            idx[i] = sorted(idx[i])
        else :
            idx[i] = data.index
            idx[i] = sorted(idx[i])
    
    return idx

def prediction_function_threshold ( model=None , X=None, Y_proba=None , seuil = 0.5 ) :
    """
    La fonction permet d'évaluer la prediction d'un modèle donnée en fonction du modèle, de l'entrée, d'une probabilité fournie et d'un seuil 
    
    Paramètres :
    ------------
        model : modèle de machine learning à utiliser.
        X : pandas.core.frame.DataFrame
            donnée à fournir au modèle pour la prédiction. Si model est fourni alors X doit aussi être fourni et dans ce cas Y_proba n'est pas utilisé.
        Y_proba : Array_type
            probabilité qu'un individu soit du label positif. Si Y_proba est fourni alors model et X ne sont pas utilisés
        seuil : float
            seuil de probalité à utiliser pour la calcul de la prediction. la valeur par defaus est 0.5 et doit toujours être comprise entre 0 et 1
    
    Return : Array_type
        prediction
        
    """
    if ( type(model) == type(None) ) and ( type(X) == type(None) ): 
        return np.array( Y_proba > seuil , dtype = int)
    else :
        try : 
            return np.array( model.predict_proba(X)[:,1] > seuil , dtype = int)
        except :
            print("Le modèle que vous avez fourni ne possède pas de méthode 'predict_proba()'")
            return
        
#  Definition de la meilleure métrique 
#  Definition de la meilleure métrique 
def my_cost( y , y_pred , poids = 4  , seuil = np.linspace(0.008,0.999,50 ) , scorer = False ) : 
    """
    La fonction permet d'évaluer le cout métier d'un modèle donné, elle évalue le cout métier pour chaque valeurs du seuil
    de probabilité
    
    Paramètres :
    ------------
        y : Array_like 
            vraie valeur labels pour chaque individus ou observations
        y_pred : Array_like 
            vecteur probabilité ( d'être positif  ) peu aussi être le vecteur prediction du modèle pour chaque individus ou observations
            
    return : dict
    ------------
        out : dict 
        La dictionnaire renvoyé est de la forme : 
        { "cout" : Array_type de variation du coût en fonction du seuil , "cout_min" : valeur du coût minimal , "seuil_min" : seuil correspondant au cout minimal }
    
    Author : 
    -----------
        Name : Brice KENGNI ZANGUIM
        e-mail : kenzabri2@yahoo.com
    
    """
    if type(y) == type(pd.DataFrame()) :
        label = pd.DataFrame( {"Y_test" : y.values.reshape( (y.shape[0],) ) , "Y_prob" : y_pred } )
    else :
        label = pd.DataFrame( {"Y_test" : y , "Y_prob" : y_pred } )

    out = {"cout" : []}
    if type(seuil) in [ float , np.float16, np.float32, np.float64] : 
        label["Y_pred"] = label["Y_prob"].apply( lambda x : int(x > seuil))
        label.loc[ (label["Y_test"] ==0) & (label["Y_pred"] == 0) , "decision"]  = "VN"
        label.loc[ (label["Y_test"] ==1) & (label["Y_pred"] == 1) , "decision"]  = "VP"
        label.loc[ (label["Y_test"] ==0) & (label["Y_pred"] == 1) , "decision"]  = "FP"
        label.loc[ (label["Y_test"] ==1) & (label["Y_pred"] == 0) , "decision"]  = "FN"
        return ( label["decision"] == "FP" ).mean() + poids*( label["decision"] == "FN" ).mean()
    else :
        for s in seuil : 
            label["Y_pred"] = label["Y_prob"].apply( lambda x : int(x > s))
            label.loc[ (label["Y_test"] ==0) & (label["Y_pred"] == 0) , "decision"]  = "VN"
            label.loc[ (label["Y_test"] ==1) & (label["Y_pred"] == 1) , "decision"]  = "VP"
            label.loc[ (label["Y_test"] ==0) & (label["Y_pred"] == 1) , "decision"]  = "FP"
            label.loc[ (label["Y_test"] ==1) & (label["Y_pred"] == 0) , "decision"]  = "FN"
            
            # Calcul du coût métier
            out["cout"].append( ( label["decision"] == "FP" ).mean() + poids*( label["decision"] == "FN" ).mean() )

        out["cout"] = np.array(out["cout"])
        # Si le cout métier est utilisé comme scorer ou métrique  alors la fonction retourne la valeur minimale pour 
        # toutes les valeurs de seuil probabilité renseignées
        if scorer : 
            return out["cout"].min()
        # Sinon la fonction revoie toutes les valeurs du cout métiers pour chaque valeurs du seuil de probabilité
        else : 
            out["cout_min"] = out["cout"].min()
            out["seuil_min"] = seuil[out["cout"] == out["cout_min"]]
            out["cout"] = list(out["cout"])
            return out
        
def print_scores(model = None , X_test=None , Y_true=None , Y_proba = None  , line_width = 6 , seuil = np.linspace( 0 , 0.9 , 90 ) ,
                 plot_kind = "apr", give_results = False , show_graph = True, poids = None, fig_sz = (12,8)) :
    scores = {} 
    beta = np.linspace( 0.7 , 2. , 2 )
    if ( type(model)!=type(None) ) and ( type(X_test)!=type(None) ) and ( type(Y_proba) == type(None) ) :
        try :
            Y_proba = model.predict_proba(X_test)[:,1]
        except :
            print("Le modèle que vous avez fourni ne possède pas de méthode 'predict_proba'")
            return
    
    ## Pour chaque coefficient beta de F_beta score, je vais calculter la variation du F_beta score avec le seuil
    if "b" in plot_kind :
        for b in beta :
            scores[f"beta = {b}"] = [  metrics.fbeta_score(Y_true , prediction_function_threshold(Y_proba = Y_proba ,seuil = s) , beta = b ) for s in seuil ]
    if "a" in plot_kind :
        scores[f"Accuracy"] = [ metrics.accuracy_score(Y_true , prediction_function_threshold(Y_proba = Y_proba ,seuil = s)  )  for s in seuil  ]      #  Accuracy en fonction du seuil
    if "r" in plot_kind :
        scores[f"Recall"] = [ metrics.recall_score(Y_true , prediction_function_threshold(Y_proba = Y_proba ,seuil = s)  ) for s in seuil  ]           #  Recall en fonction du seuil
    if "p" in plot_kind :
        scores[f"Precision"] = [ metrics.precision_score(Y_true , prediction_function_threshold(Y_proba = Y_proba ,seuil = s)  ) for s in seuil  ]     #  Precision en fonction du seuil
    if "h" in plot_kind :
        scores[f"Hamming Loss"] = [ metrics.hamming_loss(Y_true , prediction_function_threshold(Y_proba = Y_proba ,seuil = s)  ) for s in seuil  ]     #  Precision en fonction du seuil
    if "c" in plot_kind :
        scores["Fonction Coût"] = my_cost(Y_true , Y_proba , seuil = seuil ,scorer=False, poids = poids)["cout"]     #  Precision en fonction du seuil
    
    if show_graph :
        #  Affichage de la figure
        plt.figure(figsize = fig_sz)
        plt.title("\nF_beta-accuracy-Précision-Recall VS seuil" , size=2*fig_sz[0])
        plt.xlabel(" Seuil de probabilité" , size= 1.5*fig_sz[0])
        plt.ylabel("SCORE" , size = 1.5*fig_sz[0] )
        
        for label , y  in scores.items()  :
            plt.plot(seuil , y , lw = line_width ,ls = np.random.choice(["dashed","dotted", "dashdot", "solid"]), label = f"{label}")
            
        plt.legend(loc="best" , fontsize="xx-large")
        plt.show()
        scores["seuil"] = seuil
    if give_results : return pd.DataFrame(scores ).set_index("seuil")

def world_and_number_cloud_show( data, fig_size = (8,5), max_wd = 150, do_mask = False, horizontal= .85, save_name = "worcloud.png", 
                     min_font = 5, font_step= 2, save_fig = True,indice= "_") :
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        #####################  Si toutes les données sont des valeurs numériques
        if np.all( [ not isinstance(data[i], str)  for i in range(len(data))] )  :
            f = np.vectorize(lambda x : f"{indice}{x}")
            data = list(f(data).ravel())
        
        if do_mask : 
            mask = np.array(Image.open("cloud.jpg"))
            mask[mask > 1] = 255
            x = wordcloud.WordCloud(width = fig_size[0]*100, height = fig_size[1]*100, background_color ='white', colormap="plasma", max_words=max_wd,
                                    repeat = False, min_font_size=min_font, font_step=font_step, prefer_horizontal = horizontal, mask = mask,
                                    relative_scaling =0, collocations =False).generate(" ".join( data))
        else :
            x = wordcloud.WordCloud(width = fig_size[0]*100, height = fig_size[1]*100, background_color ='white', colormap="plasma", max_words=max_wd,
                                    repeat = False, min_font_size=min_font, font_step=font_step, prefer_horizontal = horizontal,relative_scaling =0,
                                    collocations =False).generate(" ".join( data))

        # plot the WordCloud image                      
        plt.figure(figsize = fig_size, facecolor = None)
        plt.imshow(x)
        plt.axis("off")
        plt.tight_layout(pad = 0)
        if save_fig : plt.savefig(save_name)
        plt.show()
        
def process_images ( data , label="images" , process_type = "MEC", resolution =( 128,128),filter_size = 3): 
    if type(data) == type(pd.DataFrame()) :
        data = data.copy()

        if "E" in process_type :
            # Uniformisation de la distributuion de pixels
            data["images_processes"] = data[label].apply(lambda image : PIL.ImageOps.equalize(image ) ) 

        if "G" in process_type :
            #picts["images_process"] = picts.images_process.apply(lambda image : img_repixels( image,  20, 220) )
            data["images_processes"] = data.images_processes.apply(lambda image : image.filter( PIL.ImageFilter.GaussianBlur(gauss_size) )  )
        if "C" in process_type :
            # Mise en contraste de l'image
            data["images_processes"] = data.images_processes.apply(lambda image : PIL.ImageOps.autocontrast( image )  )
        
        if "R" in process_type :
            data["images_processes"] = data.images_processes.apply( lambda img : np.asarray(img) )
            data['images_processes'] = data.images_processes.apply( lambda img :  img.resize( resolution, resample = PIL.Image.Resampling.BICUBIC) )

        data['images_process'] = data["images_processes"]
        data.drop( columns=["images_processes"] ,inplace = True)
        return data
    else :
        image = data
        from PIL import ImageOps
        if "C" in process_type :
            # Mise en contraste de l'image
            image = ImageOps.autocontrast( image )
        if "E" in process_type :
            # Uniformisation de la distributuion de pixels
            image =  PIL.ImageOps.equalize( image )
        if "M" in process_type :
            if filter_size%2 == 0 : filter_size += 1
            # application d'un median
            image =  image.filter( PIL.ImageFilter.MedianFilter(filter_size) )
        elif "G" in process_type :
            # Application du filtre gaussien
            # picts["images_process"] = picts.images_process.apply(lambda image : img_repixels( image,  20, 220) )
            image = image.filter( PIL.ImageFilter.GaussianBlur(filter_size) )
        if "R" in process_type :
            #image = np.asarray(image) 
            image = image.resize( resolution, resample = PIL.Image.Resampling.LANCZOS )
            #image = PIL.Image.fr(image)
        return image


def afficher_une_serie_d_images( true_img, predict_img_0, predict_img_1 = None,
                                n = 1, figsize = (16,8), gray = False, title = [" "," "," "], alpha = 1 , external_style = False) :
    
    if isinstance( true_img, list ) :
        true_img = np.array([ true_img[i] for i in  range(len(true_img))] )
        predict_img_0 = np.array( [ predict_img_0[i] for i in range(len(predict_img_0))] )
        if not isinstance( predict_img_1, type(None) ) :
            predict_img_1 = np.array( [ predict_img_1[i] for i in range(len(predict_img_1))] )
    if isinstance( true_img[0], str ) :
        true_img = [ image_utils.load_img( true_img[i], grayscale=gray)  for i in range(len(true_img)) ] 
        predict_img_0 = [  image_utils.load_img(predict_img_0[i], grayscale=gray) for i in range(len(true_img)) ] 
        if not isinstance( predict_img_1, type(None) ) :
            predict_img_1 = [  image_utils.load_img(predict_img_1[i], grayscale=gray) for i in range(len(true_img))  ] 

    if  isinstance( predict_img_1, type(None) ) : 
        X = [ true_img, predict_img_0 ] 
    else  :
        X = [ true_img, predict_img_0, predict_img_1 ] 
    ###########   Vérifier que le nombre de donées à afficher est inférieur au nombre de données disponibles ############
    assert  n <= np.array(X[0]).shape[0] 
    
    idx = np.sort( np.random.choice([i for i in range(np.array(X[0]).shape[0])], size=n , replace = False  ) )
    
    for ligne in range(n) :
        
        fig , ax = plt.subplots( 1,len(X), figsize = figsize, sharex=True )
        for j in range(len(X)) :
            if not external_style :
                ax[j].imshow( X[ j ][ idx[ ligne ] ]) 
            elif external_style == 2 :
                if j == 1  :
                    ax[j].imshow( X[j-1][idx[ligne]] ) 
                    ax[j].imshow( X[j][idx[ligne]], alpha = alpha )

                else :
                    ax[j].imshow( X[j][idx[ligne]] ) 
            elif external_style == 3 :
                if j == 2 :
                    ax[j].imshow( X[j-2][idx[ligne]] ) 
                    ax[j].imshow( X[j-1][idx[ligne]], alpha = alpha )

                else :
                    ax[j].imshow( X[j][idx[ligne]] ) 
            
            ax[j].set_title( title[j], size = 1.1*figsize[0] )
            ax[j].grid(False)
            ax[j].axis("off")

        plt.show()


def afficher_les_images_supperposees( img1, img2 = None, style = 1, figsize = (15,8), alpha = 0.4, title = None ):
    if isinstance(img1,str): 
        img1 = image_utils.load_img(img1)
        if img2 :
            img2 = image_utils.load_img(img2)
    else :
        img1, img2 = np.asarray(img1), np.asarray(img2)
    
    if style == 1 :
        plt.figure(figsize=(16,8))
        plt.imshow(img1 )
        if np.all(img2) :
            plt.imshow(img2, alpha  = alpha )
            plt.title("Image + masque", size = 1.5*figsize[0])
        else : 
            plt.title( title, size = 1.5*figsize[0])
        plt.grid(False)
        plt.axis("off")
        plt.show()
        
    elif style == 2 :
        if title  :
            afficher_une_serie_d_images([img1],[img2],n = 1, alpha = alpha, title=title, external_style=style,figsize=figsize)
        else : 
            afficher_une_serie_d_images([img1],[img2],n = 1, alpha = alpha, title=["simple image", "supperposition"], external_style=style,
                                        figsize=figsize)
            
    elif style == 3 :
        if title :
            afficher_une_serie_d_images([img1],[img2],[img2],n = 1, alpha = alpha, title=title, external_style=style,figsize=figsize)
        else :
            afficher_une_serie_d_images([img1],[img2],[img2],n = 1, alpha = alpha, title=["simple image","masque" ,"supperposition"],
                                        external_style=style,figsize=figsize)
    else :
        print("Le style d'affichage que vous avez selectionné n'est pasdisponible, veillez le changer")



def couleurs_uniques(img, return_counts = False) :
    if isinstance(img, str) :
        img = image_utils.load_img(img)
    img = np.asarray(img)
    
    if return_counts :
        tmp, count = np.unique( img.reshape( -1 , img.shape[-1] ) , axis = 0 , return_counts = return_counts )
        return [f"({i[0]},{i[1]},{i[2]})" for i in tmp ], count
    else :
        tmp = np.unique( img.reshape( -1 , img.shape[-1] ) , axis = 0 )
        return [f"({i[0]},{i[1]},{i[2]})" for i in tmp ]
    
def afficher_les_metriques( result, figsize = (15,10) ) :
    fig, ax = plt.subplots(2,2, figsize = figsize )

    ax[0,0].plot(result.index , result["Tversky_coef"])
    ax[0,0].plot(result.index , result["val_Tversky_coef"])
    ax[0,0].set_title( "Tversky_coef", size = 16 )
    ax[0,0].legend(["train","val"], loc = 'best')
    #ax[0,0].set_xlabel("Itération")

    ax[0,1].plot(result.index , result["dice_coeff"])
    ax[0,1].plot(result.index , result["val_dice_coeff"])
    ax[0,1].set_title( "dice_coeff", size = 16 )
    ax[0,1].legend(["train","val"], loc = 'best')

    ax[1,0].plot(result.index , result["mean_IoU"])
    ax[1,0].plot(result.index , result["val_mean_IoU"])
    ax[1,0].set_title( "mean_IoU", size = 16 )
    ax[1,0].legend(["train","val"], loc = 'best')

    ax[1,1].plot(result.index , result["ARI_score"])
    ax[1,1].plot(result.index , result["val_ARI_score"])
    ax[1,1].set_title( "ARI_score", size = 16 )
    ax[1,1].legend(["train","val"], loc = 'best')

    plt.show()

    
class MyCallback( tf.keras.callbacks.Callback ):
    
    def __init__(self, seuil = 0.95) :
        self.seuil = seuil
    
    def on_epoch_end(self, epoch, logs={}):
        if logs['dice_coeff'] > self.seuil and 0.99 < logs['ARI_score']  < 1. :
            print(f"\n\n La métrique dice_coeff > {self.seuil*100}; en conséquence de quoi l'entrainement est arrêté ! !\n")
            self.model.stop_training = True
        elif logs['mean_IoU'] > self.seuil and 0.99 < logs['ARI_score']  < 1.:
            print(f"\n\n La métrique mean_IoU > {self.seuil*100}; en conséquence de quoi l'entrainement est arrêté ! !\n")
            self.model.stop_training = True
        elif logs['Tversky_coef'] > self.seuil and 0.99 < logs['ARI_score']  < 1.:
            print(f"\n\n La métrique Tversky_coef > {self.seuil*100}; en conséquence de quoi l'entrainement est arrêté ! !\n")
            self.model.stop_training = True
            
def scheduler(epoch, learning_rate):
    
    if epoch %10 == 4 :
        return learning_rate*1.05
    else :
        return learning_rate

## <strong> Métriques

In [3]:
###################################################################################
##################     Dice loss et Coefficient de Dice     #######################
###################################################################################

def dice_coeff(y_true, y_pred):
    
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2.*intersection + smooth)/(K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    
    return 1.0 - dice_coeff(y_true, y_pred)

def binary_dice_entropy(y_true, y_pred):
    
    return categorical_crossentropy(y_true, y_pred) + 3*dice_loss(y_true, y_pred)
    #return CategoricalCrossentropy()(y_true, y_pred) + 3*dice_loss(y_true, y_pred)

##############################################################################
##################     Tversky loss  with beta = 0.5   #######################
##############################################################################

def Tversky_coef(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.math.sigmoid(y_pred)
    numerator = 2*y_true * y_pred
    denominator = y_true + y_pred 

    return tf.reduce_sum(numerator) / tf.reduce_sum(denominator)

def Tversky_loss(y_true, y_pred):
    return 1.0 - Tversky_coef(y_true, y_pred)

##############################################################################
##################       Intersection sur l'union      #######################
##############################################################################

def mean_IoU(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f)
    score = (intersection + 1.) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1. )
    return score

def IOU_good (y_true, y_pred ) :
    y_true = tf.cast(  y_true , tf.float32 )
    y_pred = tf.cast(y_pred, tf.float32)
    
    y_true_f = K.flatten( y_true )
    y_pred_f = K.flatten( y_pred )
    
    i = tf.cast( y_true_f == y_pred_f, tf.float32 )
    j = tf.cast( y_true_f != y_pred_f, tf.float32 )
    
    intersection = tf.reduce_sum( i )
    union = 2*tf.reduce_sum( j ) + intersection

    return intersection/union

def IoU_loss ( y_true, y_pred) :
    
    return 1.0 - mean_IoU(y_true, y_pred)

def Iou_binary_cross_entropy ( y_true, y_pred) :
    
    return categorical_crossentropy( y_true, y_pred ) + 3*IoU_loss( y_true, y_pred )
    #return CategoricalCrossentropy()( y_true, y_pred ) + 3*IoU_loss( y_true, y_pred )



##############################################################################
##################          Segmentation ARI           #######################
##############################################################################

def ARI_score ( y_true, y_pred ) :
    """
    Implementation du score Adjusted Rand score des auteurs Lawrence Hubert  et Phipps Arabic publié dans Journal of Classification en 1985 
    - https://en.wikipedia.org/wiki/Rand_index 
    - Equation 5 de l'article : https://pdfslide.net/documents/lawrence-hubert-and-phipps-arabie-comparing-partitions-1985.html?page=1
    
    Le score permet de mesurer la similitude entre deux partition d'un memê ensemble. ici je l'adpate pour l'évaluation des performances 
    d'une segmentation d'images. la même métrique est disponible sous scikit learn 
    ( https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html) mais s'avère impuissante pour des entrées 
    `y_true` et `y_pred` de type Tensorielle comme celles qui peuvent être fournies par la sortie d'un réseau de convolution.
    
    Liberté vous est donnée de de copier, partager, modifier, améliorer le code à votre gré.
    Paramètres :
    ------------
        y_true, y_pred : Tensor de la forme ( hauteur, largeur , canal ): 
      Auteur :
    -----------
        - Nom     :  Brice KENGNI ZANGUIM
        - E-mail  :  kenzabri2@yahoo.com
    """
    ####################################################################
    ####     Transformation des entrées en un tenseur d'ordre 1    #####
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    #if tf.math.all( y_true_f == y_pred_f ) :
    #    return 1.
    
    nij = tf.math.confusion_matrix(y_true_f,y_pred_f) 
    
    a_i = tf.reduce_sum(nij, axis=0)
    b_j = tf.reduce_sum(nij, axis=1)
    
    a_i_sum = tf.reduce_sum(a_i * (a_i - 1), axis=0)
    b_i_sum = tf.reduce_sum(b_j * (b_j - 1), axis=0)
    n = tf.size(y_pred_f)

    RI = tf.reduce_sum(nij * (nij - 1), axis=(0,1) )
    max_RI = ( a_i_sum + b_i_sum ) / 2
    expected_RI = a_i_sum * b_i_sum / n
    
    RI = tf.cast(RI, tf.float32)
    expected_RI = tf.cast(expected_RI, tf.float32)
    max_RI = tf.cast(max_RI, tf.float32)
    
    return (RI - expected_RI) / (max_RI - expected_RI)

def ARI_loss( y_true ,y_pred ) :
    
    return 1 - ARI_score(y_true, y_pred)

##############################################################################
##################           Entropie mixte            #######################
##############################################################################

def mixt_entropi(y_true =None , y_pred=None, iou_coef=1.4, twersky_coef=1.4,dice_coef=1.2, ari_coef = 0.04) :
    
    IOU = iou_coef*IoU_loss(y_true, y_pred)
    TVERSKY =  twersky_coef*Tversky_loss(y_true, y_pred)
    DICE =  dice_coef*dice_loss( y_true,y_pred )
    ARI = ari_coef*ARI_loss( y_true, y_pred )
    
    return categorical_crossentropy(y_true, y_pred) + IOU + TVERSKY + DICE + ARI
    #return CategoricalCrossentropy()(y_true, y_pred) + IOU + TVERSKY + DICE + ARI

## <strong> Mes classes personnalisées

- ### <strong> Transformer de masques

In [4]:
class Mask_To_8_Groups(BaseEstimator,TransformerMixin):
    
    """
        Description :
        ---------------
        
        Cette classe prends transforme une matrice de taille (n,m) en un tenseur de configuration (n,m,k) où k est le nombre de catégories 
        suivant la troisième dimension k, les composantes sont des vecteurs qui One hot Encode les pixels présents sur la matrice de départ
        
        D'une certaine façon celà équivaut à un One hot Encode des différentes valeurs de pixels présents sur la matrice de départ d'où la
        troisième dimension k qui a pour valeur le nombre de categories
        
        Author :
        -----------
            Name : Brice KENGNI ZANGUIM
            e-mail : kenzabri2@yahoo.com
    """
    ########################################################################################################
    #############        catégorie de groupage par defaut. L'utilisateur peut  fournir         #############
    ############# une autre catergorisation de groupage lors de l'instantiation de la Classe   #############
    categorie_par_defaut =  {
                    'void': [0, 1, 2, 3, 4, 5, 6],
                    'flat': [7, 8, 9, 10],
                    'construction': [11, 12, 13, 14, 15, 16],
                    'object': [17, 18, 19, 20],
                    'nature': [21, 22],
                    'sky': [23],
                    'human': [24, 25],
                    'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]
                }
    
    #############################################################################################################
    #############  Palette de couleurs par defaut pour les classes si l'on veut que chaque classe   #############
    #############  soit représentée par une couleur bien spécifique en RGB. Les couleurs doivent    #############
    #############    suivre le même ordre de definition que dans le dictionnaire de catégories      #############
    palette_couleur_par_defaut =[ 'ivory', 'lightgrey', 'plum', 'olive', 'forestgreen', 'skyblue', 'orangered', 'navy']
    
    def __init__( self, categorie = None, palette_couleur = None):
        ############################################################################################################
        #############  Si une categorisation est fournie lors de l'instantiation alors je l'utilise    #############
        if categorie :
            assert isinstance(categorie, dict)
            self.categorie = categorie
        
        ###################################################################################
        #############  Sinon j'utilise plutot une categorisation par defaut   #############
        else :
            self.categorie = Mask_To_8_Groups.categorie_par_defaut
        self.nom_des_categories = self.categorie.keys()
        self.numero_des_categories = np.array(range(len(self.categorie)) )
        if np.all(palette_couleur) :
            self.palette_couleur = palette_couleur
            assert len(self.palette_couleur) == len(self.categorie)
        else :
            self.palette_couleur = Mask_To_8_Groups.palette_couleur_par_defaut
        

    def fit( self, categorie, palette_couleur = None ):
        self.categorie = categorie
        self.nom_des_categories = categorie.keys()
        self.numero_des_categories = np.array(range(len(self.categorie)) )
        
        if np.all( palette_couleur ) :
            self.palette_couleur = palette_couleur
            assert len(self.palette_couleur) == len(self.categorie)

    
    def transform(self ,img ,y = None ):
        img = np.array( img ) 
        
        #############################################################################
        ############# Initialisation d'un masque de taille (n ,m ,k )   #############
        mask = np.zeros( (img.shape[0], img.shape[1], len( self.nom_des_categories ) ) ) 
        
        ####################################################################################################
        ############# je parcours toute la liste des identifiants `Id` des différents objets   #############
        for Id in range(-1, 34):
            ####################################################################################################
            #############     Je parcours toute la liste des noms de catégories d'objets afin de    ############
            #############   rechercher la catégorie qui associée à l'Id et créer le canal associé   ############
            for pos, cat in enumerate(self.nom_des_categories) :
                if Id in self.categorie[cat]:
                    #mask[:, :, pos] = np.logical_or(mask[:, :, pos], img == Id )
                    mask[:, :, pos][img == Id] = 1
                    break

        return mask
    
    def fit_transform ( self ,img ,categorie = None )  :
        if categorie :
            self.fit( categorie )
        
        return self.transform(img)
    
    def invers_transform( self, mask ) :
        """
        Prends le masque en entrée et reconstruit l'image d'origine au format noir sur blanc
        La petite nuance est qu'il y a à présents autant de niveaux de pixels que de groupes associés à la transformation
        l'image n'est donc pas ramenée au 34 groupes initiaux qui apparaissaient sur l'image originale mais au nombre de 
        groupes réduis associés à la transformation; soit 8 groupes pas defaut.  Pour remontrer aux 34 groupes de départ 
        il faurdrait une astuce plus ingénieuse.
        """
        
        #img = np.zeros((mask.shape[0], mask.shape[1]))
        ###############################################################################################################
        ################  Je me crèe un vectoriser qui donne le pouvoir à une fonction de pouvoir          ############ 
        ################  se transformer en une fonction vectorielle multidimension definie de E vers E    ############ 
        #f = np.vectorize(lambda x: self.numero_des_categories[x] )
        
        ############################################################################################################################
        ####################  Ma fonction vectorielle prends toutes les indexes où se trouvent les maximum              ############ 
        ####################  dans la troisième dimension et y retourne la position le numero de categori  équivalent   ############ 
        if mask.ndim == 3 and mask.shape[-1] == 8 : 
            return np.argmax(mask, axis=2)    # Ceci tire profit du fait que les numéro de catégories sont ordonnés à partir de 0
        
        elif mask.ndim == 3 and mask.shape[-1] == 3  :  # Images au format RGB
            output_mask = np.zeros(mask.shape[0]* mask.shape[1])
            x = mask.reshape((-1,3))
            for classe, couleur in enumerate(self.palette_couleur) :
                output_mask[ np.prod(x == colors.to_rgb( couleur ), axis = 1, dtype=bool) ] = classe
            
            return output_mask.reshape((mask.shape[0],mask.shape[1] ))
        
        #return  f( np.argmax(mask, axis=2)  )
        
    def fit_transform_to_specific_color_set(self,mask, palette_couleur = None ) :
        if palette_couleur : self.palette_couleur = palette_couleur
        
        ##################################################################################################################
        ###############  J'initialise à zéro tous les éléments de l'image RGB qui seras renvoyé en sorti    ##############
        img_rgb = np.zeros((mask.shape[0], mask.shape[1], 3), dtype= float)
                        
        if np.ndim( mask ) == 3 :
            ###################################################################################################################
            ###########    Avant toute chose on se rassure que la profondeur du masque ( sa troisième dimenssion)     #########
            ###########          a la même longueur que le dictionnaires des classes (  nombre de classe )            #########
            # assert mask.shape[-1] == len( self.categorie ) # Uniquement que si la fonction invers_transform ne prends pas les images RGB
            
            ##############################################################################################################
            ###########    Je ramène mon masque de 3 dimensions à un masque de 2 dimension (niveau de gris )     #########
            mask = self.invers_transform(mask)
        
        assert np.ndim(mask) == 2
        for cats in np.unique(mask) :
            for i in [0,1,2] :
                img_rgb[:,:,i][ mask == cats ] = colors.to_rgb( self.palette_couleur[cats] )[i]
        
        return img_rgb
        
        ###Pour chaque cétégorie, donne la valeur de la couleur du pixel (pour R, G et B)
        ###for cat in range(len(self.categorie)):
        ###    for i in [0,1,2] :
        ###        img_rgb[:, :,i] += mask[:, :, cat] * colors.to_rgb(colors_palette[cat])[i]
            

- ### <strong> Générateur d'images

In [5]:
class MyImagesGenerator ( keras.utils.Sequence ):
    
    default_image_size = (128,256)
    
    #####################################################################################################################################
    #####      Essentiellement utile lorsque les données d'entrainement de teste et de validation sont dans le même dossier        ######
    #####  Les variables de classe serviront à faire la séparation entre les données train/test/validation dans un seul dossier    ######
    nombre_d_instances = 0
    fichiers_libres = []
    
    def __init__( self, chemin_image, chemin_masque, batch_taille, shuffle =False, taille_des_donnees = None,  augmentation = False, 
                   aug_batch_size = 2, aug_nb_batch = 2, cats = None, mask_transformer=None, target_size = None): 
        """
            La fonction permet d'initialiser les attributs d'instances de l'instance de classe nouvellement déclarée

            Paramètres :
            ---------------
                chemin_image, chemin_masque  : Str 
                    Chemin d'accès aux images et aux masques associés. Il est important de terminer la définition des chemin par le caractère "/"
                    Si celà n'est pas respecté le programme effectue une correction automatique sur les chemins concernés.
                batch_taille : int 
                    Taille d'un paquet de données (batch)
                shuffle : Boolean
                    Grandeur booléenne qui permet de spécifier si on doit modifier l'ordre de rangement des données qui seront transmises au modèle
                    de Deep Learning. Les valeurs admises sont "True" et "False"
                target_size : tuple
                    Un tuple qui défini les dimensions des images si elles doivent être redimensionnés ou non. Si aucune valeur n'est spécifiée
                    alors les images sont utilisées sans être redimensionnées
                cats : dict
                    dictionnaire des catégories de regroupement des classes si ce dernier. S'il n'est pas foruni alors la classification utilisée
                    par defaut est celle en 8 groupes spécifié sur le site https://www.cityscapes-dataset.com/dataset-overview/
                    Voire la classe Mask_To_8_Groups() pour plus de détails.
                taille_des_donnees : int
                    Nombre de couple ( image , masque ) a retenir parmi la liste disponible dans les dossiers indiqués par les chemins
                augmentation : Bool
                    Précise si une augmentation des données doit être faite ou non
                aug_batch_size, aug_nb_batch : int
                    respectivement la taille et le nombre de batch à générer dans la classe d'augmentation de données afin de réduire le temps de
                    génération des images et masques augmentés
                    
            return : dict
            ---------------
                None

            Author : 
            -----------
                Name : Brice KENGNI ZANGUIM
                e-mail : kenzabri2@yahoo.com
        """
        #######################################################################################
        ######################  Incrément du nombre d'instances créés  ########################
        MyImagesGenerator.nombre_d_instances += 1
        
        #########################################################################
        ######################  Préparation des données  ########################
        if chemin_image[-1] != "/" : chemin_image += "/"
        if chemin_masque[-1] != "/" : chemin_masque += "/"
        
        ######################################################################################################
        ########################  Il faut vérifier que les masques et les images ne ##########################
        ########################       sont pas logés dans le même repertoire     ############################ 
        assert chemin_image != chemin_masque , "Veillez spécifier des chemin d'accès différents pour les images et les masques"
        self.chemin_image = chemin_image
        self.chemin_masque = chemin_masque
        
        #####################################################################################################
        #############  Les fichier correspondants images et masque doivent avoir le même nom  ###############
        #############  On se rassure que les fichiers masques et images on les mêmes noms     ###############
        if MyImagesGenerator.nombre_d_instances == 1 : 
            # On initialise la liste des fichiers libre et on on se rassure que les noms de fichiers sont iddentiques dans les deuc chemins
            MyImagesGenerator.fichiers_libres = np.array( [ i for i in os.listdir(self.chemin_image) if ".png" in i] )
            assert sorted(MyImagesGenerator.fichiers_libres) == sorted([ i for i in os.listdir(self.chemin_masque) if ".png" in i] )

        ####################################################################################################
        ###############     Definition de la liste de fichier propres à l'instance créée      ##############
        assert len(MyImagesGenerator.fichiers_libres) >= taille_des_donnees , "Le nombre de données demandée est suppérieur au nombre disponible"
        self.files_name = np.random.choice( MyImagesGenerator.fichiers_libres , taille_des_donnees, replace = False )

        #######################################################################################################
        ###############             Mise à jour de la liste de fichiers disponibles              ##############
        MyImagesGenerator.fichiers_libres = np.array( [ files 
                                                        for files in MyImagesGenerator.fichiers_libres  
                                                        if files not in self.files_name  
                                                      ] )           
            
        ########################################################################################
        ###############         Definition des indexes d'accès aux données        ##############
        self.indexes = np.arange(len(self.files_name))
            
        
        ######################################################
        #################      DIVERS      ###################
        self.batch_taille = batch_taille
        self.shuffle = shuffle
        self.augmentation = augmentation
        self.aug_batch_size = aug_batch_size
        self.aug_nb_batch = aug_nb_batch
        
        if target_size : 
            self.target_size = target_size
        else :
            self.target_size = MyImagesGenerator.default_image_size
        
        ###############################################################################
        ###############   Transformer pour la transformation du masque   ##############
        if mask_transformer : 
            self.mask_transformer = mask_transformer
        else :
            self.mask_transformer = Mask_To_8_Groups( Mask_To_8_Groups.categorie_par_defaut )
        
        ################################################################################################
        ##############     Je lance un premier appel à la fonction on_epoch_end() pour     #############
        ##############    randomiser l'ordre des données, ceci en créant de l'aléatoire    #############
        ##############     à l'ordre de primaire de la variable d'instance "indexes"       #############
        self.on_epoch_end()
        
    def __len__(self) :
        """
        Permet de définir le nombre de batch qui seront transmis au modèle durant l'itération. Le calcul se base sur la taille totale 
        des données et la taille d'un batch.
        
        En un sens, si j'ai 100 images à passer à mon modèle durant une itération et que la taille du batch est de 3 alors on aura en tout
        100//3 batchs ou paquets de 3 items chacun qui seront constitués et envoyés progressivement au modèle 
        """
        ###################################################################################################################################
        ############## Je retourne le nombre de requête de batch qui doivent être effectués à chaque itération ou epoch  ##################
        return int( np.ceil( len(self.files_name)/self.batch_taille ) )
        
    def __getitem__(self, index) :
        """
        C'est cette fonction qui est appelée chaque fois que necessaire lorsque le modèle a besoin d'un nouveau batch de donnée
        Elle est appelée avec un argument "index" qui spécifie l'ordre du batch. Par exemple lors du premier appel de la fonction pendant 
        l'entrainement du modèle, index prends la valeur 0, pour le second batch index prends la valeur 1 et ainsi de suite.
        """
        if index == self.__len__() - 1 : 
            position_dans_la_liste_de_donnees = self.indexes[ index*self.batch_taille : ]
        else :
            position_dans_la_liste_de_donnees = self.indexes[ index*self.batch_taille : (index+1)*self.batch_taille ]
        
        #images, mask = [], []
        #for idx in position_dans_la_liste_de_donnees :
        #     images.append( np.array( image_utils.load_img( self.chemin_image+ self.files_name[idx] ) ) )
        
        ###############################################################################################
        ############  Je charge les images, ensuite je les transforme en une matrice    ###############
        ############  np.array() et pour finir j'ajoute le tout à la liste des images   ###############
        images = [  np.array( 
                              image_utils.load_img( self.chemin_image + name, target_size = self.target_size,
                                                   interpolation="lanczos"  ) , dtype= 'uint8'
                             )  
                      for name in self.files_name[ position_dans_la_liste_de_donnees ] 
                 ]

        ################################################################################################
        ############  Je charge les masques, ensuite je les transforme en une matrice    ###############
        ############  np.array() et pour finir j'ajoute le tout à la liste des images    ###############            
        masks = [   np.array( 
                              image_utils.load_img(self.chemin_masque + name, grayscale = True,
                                                   target_size = self.target_size, interpolation="lanczos" ) , dtype = 'uint8'
                             )  
                        for name in self.files_name[ position_dans_la_liste_de_donnees ]
               ]
        
        ########################################################
        ############  Augmentation des images    ###############
        if self.augmentation : 
            augmenter = AugmentationImagesEtMask( images , masks, batch_size = self.aug_batch_size , nb_batch = self.aug_nb_batch ,  )
            images, masks = augmenter.transform()
        
        #################################################################
        ############  encodage des masque en 8 classes    ###############
        masks = [  self.mask_transformer.transform( i )   for i in masks ]        
        
        ##########################################################################################
        ########   Je remodifie l'ordre des données du batch et les renvoie au modèle   ##########
        x = np.arange(len(images))
        np.random.shuffle(x)       # mélangeur
        return np.array(images)[x] , np.array(masks)[x]
        
    def on_epoch_end( self ) :
        """
        Cette fonction est appelée à la fin de chaque itération.
        Elle peut entre autre permettre de définir la façon dont les données seront transmises aux modèles. Supposons que lors de l'itération actuelle
        les items 1 et 3 faisaient parti d'un même batch, on peut s'arranger pour que lors de l'itération suivante ces deux items soient transmis au 
        modèle dans deux batch distinct; par exemple l'item 1 étant trans mis via le deuxième batch et l'item 3 est transmis via le dixième batch.
        
        Cette opération est rendue possible modifiant l'ordre dans lequel les items sont rangés entre deux itération; un peu comme si on mélangéait
        des cartes dans un jeux de cartes. Celà permet de réduire les effets du batch sur les résultats de l'entrainement du modèle.
        
        La fonction trouve surtout son importance lorsqu'il faut
        """
        if self.shuffle  :
            np.random.shuffle(self.indexes)


- ### <strong> Augmenteur d'images et de masques dans les données

In [6]:
class AugmentationImagesEtMask (  ) :
    
    # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
    # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second
    # image.
    sometimes = lambda aug: iaa.Sometimes(0.3, aug)

    # Define our sequence of augmentation steps that will be applied to every image.
    transformation_par_defaut = iaa.Sequential(
                                                [   # Apply affine transformations to some of the images
                                                    # - scale to 80-120% of image height/width (each axis independently)
                                                    # - translate by -10 to +10 relative to height/width (per axis)
                                                    # - rotate by -45 to +45 degrees
                                                    # - shear by -16 to +16 degrees
                                                    # - order: use nearest neighbour or bilinear interpolation (fast)
                                                    # - mode: use any available mode to fill newly created pixels
                                                    #         see API or scikit-image for which modes are available
                                                    # - cval: if the mode is constant, then use a random brightness
                                                    #         for the newly created pixels (e.g. sometimes black,
                                                    #         sometimes white)
                                                    sometimes(iaa.Affine(
                                                        scale={"x": (0.9, 1.1), "y": (0.95, 1.05)},
                                                        translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)},
                                                        rotate=(-7, 7),
                                                        shear=(-4, 4),
                                                        order=[0, 1],
                                                        cval=(0, 255),
                                                        mode=imgaug.ALL
                                                    )),
                                                    
                                                   # Execute 0 to 1 of the following (less important) augmenters per
                                                    # image. Don't execute all of them, as that would often be way too
                                                    # strong.
                                                    #
                                                    iaa.SomeOf((0, 1),
                                                        [   # Blur each image with varying strength using
                                                            # gaussian blur (sigma between 2.0 and 5.0),
                                                            # average/uniform blur (kernel size between 2x2 and 7x7)
                                                            # median blur (kernel size between 3x3 and 11x11).
                                                            iaa.RemoveSaturation((0.0, 1.0)),
                                                            
                                                            # Sharpen each image, overlay the result with the original
                                                            # image using an alpha between 0 (no sharpening) and 1
                                                            # (full sharpening effect).
                                                            iaa.Sharpen(alpha=(0, 1.0), lightness=(1., 1.8)),

                                                            # Change brightness of images (70-180% of original value).
                                                            iaa.Multiply((.85, 1.8), per_channel=0.5),
                                                            
                                                            sometimes( iaa.AdditiveGaussianNoise( loc=0, scale=(0.0, 0.007 * 155), per_channel=0.2) ),
                                                            
                                                            iaa.Snowflakes(flake_size=(0.08, 0.1), speed=(0.01, 0.05)),
                                                            
                                                            iaa.Clouds() ,
                                                            
                                                            #sometimes(iaa.FastSnowyLandscape( lightness_threshold=(100, 255), lightness_multiplier=(1.0, 2.0)) ),
                                                            # crop some of the images by 0-7% of their height/width
                                                            sometimes(iaa.Crop(percent=(0, 0.007))),

                                                            iaa.SaltAndPepper(
                                                                (0.0, 0.005)
                                                            ), 
                                                             ## add rain
                                                            sometimes(  iaa.Rain(drop_size=(0.001), speed=(0, 0.01)) ),
                                                            

                                                        ],
                                                        # do all of the above augmentations in random order
                                                        random_order=True
                                                    )
                                                ],
                                                # do all of the above augmentations in random order
                                                random_order=True
                                            )

    def __init__ ( self, image, mask, batch_size=3, nb_batch=4, transformations = None) :
        if not isinstance( image,  list) :
            self.image = [image]
            self.mask = [mask]
        else : 
            self.image = image
            self.mask = mask
                    
        #######################################################################
        ########   Utile pour le mode augmentation multiprocesseur    #########
        self.batch_size = batch_size
        self.nb_batch = nb_batch
        
        #############################################################################################################
        ########   Objet definissant l'ensemble des transformations à appliquer sur l'image et le masque    #########        
        if transformations :
            self.transformations = transformations
        else :
            self.transformations = AugmentationImagesEtMask.transformation_par_defaut
        
        ##############################################################
        #######   Sortie des résultats des transformations   #########
        # return self.transform()  # Opération strictement interdite dans la fonction __init__()
        
    def transform(self) :
        #################################
        imgaug.seed( np.random.randint(1,20) )
        
        img_finale, mask_final  =[], []
        for image, mask in zip( self.image, self.mask ) :
            ################################################################################################
            ######      Objet masque représentant une association entre une image et son masque      #######
            segmap = SegmentationMapsOnImage( mask, shape = image.shape )
        
            ##############################################################################################################################
            ######    Initialisation d'un batch de masques avec des copies de masques en autant de fois quela taille du batch      #######          
            segmaps = [segmap for _ in range( self.batch_size ) ]        
        
            ############################################################################################################################
            ######    Initialisation d'un batch d'images avec des copies de l'image en autant de fois quela taille du batch      #######        
            images = [ image for _ in range( self.batch_size ) ]
        
            #######################################################################################################################
            ######    Création de notre liste de batchs à partir des deux batch images et masques précédement definies      #######         
            batches = [ UnnormalizedBatch(images=images, segmentation_maps=segmaps) for _ in range(self.nb_batch) ]
        
            ########################################################################################################################
            #########        transformation des images - Le paramètre `background` active le mode multiprocess            ########## 
            batches_aug = list( self.transformations.augment_batches( batches, background = False ) )

            img_finale.extend([ batches_aug[j].images_aug[i] for i in range(self.batch_size) for j in range(self.nb_batch) ])
        
            #################################################
            ######    transformation des masques      ####### 
            mask_final.extend([  batches_aug[j].segmentation_maps_aug[i].get_arr() 
                              for i in range(self.batch_size) 
                              for j in range(self.nb_batch)
                         ])
        
            ##############################################################################################################################
            ######    rajout des originaux des images et masques en bout de liste des images et masques augmentés/transformés      ####### 
        
            img_finale.append( image )
            mask_final.append( mask )
        
        return img_finale, mask_final